# Factorizations and other fun
Based on work by Andreas Noack

## Outline
- Factorizations
- Special matrix structures
- Generic linear algebra

Before we get started, let's set up a linear system and use `LinearAlgebra` to bring in 
the factorizations and special matrix structures.

In [1]:
using LinearAlgebra
A = rand(3, 3)
x = fill(1, (3,))
b = A * x

3-element Array{Float64,1}:
 0.4606327857425021
 1.1941853669672986
 1.037874155200157

## Factorizations

#### LU factorizations
In Julia we can perform an LU factorization
```julia
PA = LU
```
where `P` is a permutation matrix, `L` is lower triangular unit diagonal and `U` is uppser
triangular, using `lufact`.

Julia allows computing the LU factorization and defines a composite factorization type for
storing it.

In [2]:
Alu = lu(A)

LU{Float64,Array{Float64,2}}
L factor:
3×3 Array{Float64,2}:
 1.0       0.0        0.0
 0.135944  1.0        0.0
 0.117214  0.0942968  1.0
U factor:
3×3 Array{Float64,2}:
 0.440834  0.223195  0.530157
 0.0       0.786701  0.0888301
 0.0       0.0       0.238097

In [3]:
typeof(Alu)

LU{Float64,Array{Float64,2}}

The different parts of the factorization can be extracted by accessing their special
properties

In [4]:
Alu.P

3×3 Array{Float64,2}:
 0.0  1.0  0.0
 0.0  0.0  1.0
 1.0  0.0  0.0

In [5]:
Alu.L

3×3 Array{Float64,2}:
 1.0       0.0        0.0
 0.135944  1.0        0.0
 0.117214  0.0942968  1.0

In [6]:
Alu.U

3×3 Array{Float64,2}:
 0.440834  0.223195  0.530157
 0.0       0.786701  0.0888301
 0.0       0.0       0.238097

JUlia can dispatch methods on factorization objects.

For example, we can solve the linear system using either the original matrix or the 
factorization object.

In [7]:
A\b

3-element Array{Float64,1}:
 1.0000000000000002
 1.0
 0.9999999999999999

In [8]:
Alu\b

3-element Array{Float64,1}:
 1.0000000000000002
 1.0
 0.9999999999999999

Similarly, we can calculate the determinant of `A` using either `A` or the factorization
object

In [10]:
det(A) ≈ det(Alu)

true

#### QR factorizations

In Julia we can perform a QR factorization
```
A=QR
```

where `Q` is unitary/orthogonal and `R` is upper triangular, using `qrfact`.

In [11]:
Aqr = qr(A)

LinearAlgebra.QRCompactWY{Float64,Array{Float64,2}}
Q factor:
3×3 LinearAlgebra.QRCompactWYQ{Float64,Array{Float64,2}}:
 -0.11537    0.0780759  -0.990249
 -0.984269  -0.143274    0.103377
 -0.133806   0.986599    0.0933773
R factor:
3×3 Array{Float64,2}:
 -0.447879  -0.340586  -0.578951
  0.0        0.78195    0.106883
  0.0        0.0       -0.235776

Similarly to the LU factorization, the matrices `Q` and`R` can be extracted from the QR
factorizatio object via

In [12]:
Aqr.Q

3×3 LinearAlgebra.QRCompactWYQ{Float64,Array{Float64,2}}:
 -0.11537    0.0780759  -0.990249
 -0.984269  -0.143274    0.103377
 -0.133806   0.986599    0.0933773

In [13]:
Aqr.R

3×3 Array{Float64,2}:
 -0.447879  -0.340586  -0.578951
  0.0        0.78195    0.106883
  0.0        0.0       -0.235776

#### Eigendecompositions

The results from eigendecompositions, singular value decompositions, Hessenberg
factorizations, and Schur decompositions are all stored in `Factorization` types.

The eigendecomposition can be computed

In [17]:
Asym = A + A'
AsymEig = eigen(Asym)

Eigen{Float64,Float64,Array{Float64,2},Array{Float64,1}}
values:
3-element Array{Float64,1}:
 -0.9769434234784926
 -0.10966861215134172
  1.9581497766666025
vectors:
3×3 Array{Float64,2}:
  0.114894   0.93715   0.329469
 -0.705558  -0.156486  0.691159
  0.699277  -0.311869  0.643234

The values and the vectors can be extracted from the Eigen type by special indexing

In [18]:
AsymEig.values

3-element Array{Float64,1}:
 -0.9769434234784926
 -0.10966861215134172
  1.9581497766666025

In [19]:
AsymEig.vectors

3×3 Array{Float64,2}:
  0.114894   0.93715   0.329469
 -0.705558  -0.156486  0.691159
  0.699277  -0.311869  0.643234

Once again, when the factorization is stored in a type, we can dispatch on it adn write
specialized methods that exploit the properties of the factorization, e.g. that
$A^{-1}=(V\Lambda V^{-1})^{-1}=V\Lambda^{-1}V^{-1}$.

In [20]:
inv(AsymEig)*Asym

3×3 Array{Float64,2}:
  1.0           7.10543e-15  -9.76996e-15
 -8.18789e-16   1.0          -5.27356e-16
 -3.16414e-15  -5.32907e-15   1.0

## Special matrix structures
Matrix structure is very important in linear algebra. To see *how* important it is, let's
work with a larget linear system

In [21]:
n = 1000
A = randn(n, n)

1000×1000 Array{Float64,2}:
 -0.2105     0.690911    0.0280421  …  -0.40948    -0.735993   -0.305073
  0.42116    0.282775   -0.927357      -0.0961538   0.306781    0.333317
  1.21697   -0.0752656  -1.94365        1.729      -1.33497    -0.970065
  0.587659  -0.218256    1.21662       -0.69356    -0.154198    0.817533
  0.565494   1.90336    -0.765603      -2.09256     1.34165     0.465241
  0.650137   1.64735    -1.50646    …   0.0322347   0.187132   -1.0564
 -0.400634   0.699374   -0.563624       0.268268   -0.830275    0.407389
  2.21067   -0.758621   -1.26061       -0.321963    1.53824     0.597627
 -0.799919   0.484509   -1.29577        2.08704     0.617655   -2.05338
  1.54291    0.452194   -0.721794       1.4263      0.762325   -2.18277
  1.17783   -0.238487    1.16121    …   1.17105     1.6851     -0.913216
 -0.215922  -0.495221   -2.73871        0.654477    1.47817     1.16613
  1.42271    1.18497    -0.604203       1.20841     0.569446   -0.282851
  ⋮                         

Julia can often infer special matrix structure

In [22]:
Asym = A + A'
issymmetric(Asym)

true

but sometimes floating point error might get in the way.

In [23]:
Asym_noisy = copy(Asym)
Asym_noisy[1,2] += 5eps()

1.1120709456974636

In [24]:
issymmetric(Asym_noisy)

false

Luckily we can declare structure explicitly with, for example, `Diagonal`, `Triangular`,
`Symmetric`, `Hermitian`, `Tridiagonal` and `Symtridiagonal`.

In [25]:
Asym_explicit = Symmetric(Asym_noisy)

1000×1000 Symmetric{Float64,Array{Float64,2}}:
 -0.421001   1.11207     1.24501   …  -1.69326     0.873282   -1.06334
  1.11207    0.56555    -1.00262       0.262666   -1.33951     0.351696
  1.24501   -1.00262    -3.88729       1.90243    -2.44065    -0.214689
 -0.467957  -0.0807692   0.288849     -1.02258    -0.521146    1.05898
  2.62082    2.99861    -0.708128     -3.28586     0.815212   -0.776936
  0.27378    1.41634    -0.936489  …   0.506499    0.156327   -0.889031
 -0.30084    1.1133     -1.15133       0.46612    -1.09929     0.806152
  2.04773   -1.92828    -2.41109      -0.368349    1.53299     2.04258
 -1.04282    2.05582    -2.09902       2.93154     1.98697    -3.0797
  1.92111   -0.668241   -2.53191       1.39591     1.27704    -2.80468
  1.65509    0.445716    1.54423   …   0.486645    2.9252     -1.70175
 -0.497576  -0.616557   -2.01176       0.710395    1.27049     0.253761
  2.49077    0.0961056   0.106468      2.34851     1.54613    -0.248951
  ⋮                     

Let's compare how long it takes Julia to compute the eigenvalues of `Asym`, `Asym_noisy`,
and `Asym_explicit`

In [26]:
@time eigvals(Asym)

  0.220003 seconds (168.06 k allocations: 16.063 MiB, 24.22% gc time)


1000-element Array{Float64,1}:
 -88.87855326630375
 -88.35854853140344
 -87.68139016918622
 -87.03291788599472
 -86.12750825329529
 -85.56115275935016
 -85.2186456395332
 -84.93673819183122
 -84.46384808158949
 -83.9010605476811
 -83.6370493026508
 -82.90307578795239
 -82.69950122550239
   ⋮
  82.74966968566332
  83.23570128858833
  83.54812577452626
  84.22781946566788
  84.29657944874221
  84.45476861198196
  85.89302616850334
  86.12689386764114
  86.6809000170313
  86.98688441077704
  87.97591296382862
  89.20913681084022

In [27]:
@time eigvals(Asym_noisy)

  0.452948 seconds (14 allocations: 7.921 MiB)


1000-element Array{Float64,1}:
 -88.8785532663048
 -88.35854853140374
 -87.68139016918917
 -87.03291788599566
 -86.12750825329675
 -85.5611527593505
 -85.21864563953417
 -84.9367381918315
 -84.4638480815898
 -83.90106054768158
 -83.63704930265048
 -82.90307578795226
 -82.69950122550291
   ⋮
  82.7496696856632
  83.23570128858857
  83.54812577452591
  84.22781946566694
  84.29657944874128
  84.45476861198262
  85.89302616850344
  86.12689386764124
  86.68090001703303
  86.98688441077806
  87.97591296382905
  89.20913681084107

In [28]:
@time eigvals(Asym_explicit)

  0.065175 seconds (6.70 k allocations: 8.324 MiB)


1000-element Array{Float64,1}:
 -88.87855326630458
 -88.35854853140381
 -87.68139016918693
 -87.03291788599378
 -86.12750825329526
 -85.56115275934964
 -85.21864563953275
 -84.93673819183181
 -84.46384808158963
 -83.90106054768061
 -83.6370493026514
 -82.90307578795289
 -82.6995012255026
   ⋮
  82.74966968566368
  83.23570128858897
  83.54812577452631
  84.22781946566731
  84.2965794487416
  84.45476861198233
  85.893026168503
  86.12689386764103
  86.68090001703179
  86.98688441077653
  87.9759129638287
  89.20913681084099

In this example, using `Symmetric()` on `Asym_noisy` made our calculations about `5x` more
efficient :)

#### A big problem
Using the `Tridiagonal` and `Symtridiagonal` types to store tridiagonal matrices makes it
possible to work with potentially very large tridiagonal problems. The following problem
would not be possible to solve on a laptop if the matrix had to be stored as a (dense)
`Matrix` types.

In [30]:
n = 1_000_000
A = SymTridiagonal(randn(n), randn(n-1))
@time eigmax(A)

  0.591469 seconds (470.10 k allocations: 206.678 MiB, 1.43% gc time)


6.5659533952760505

## Generic linear algebra
The usual way of adding support for numerical linear algebra is by wrapping BLAS and
LAPACK subroutines. For matrices with elements of `Float32`, `Float64`, `Complex{Float32}`
or `Complex{Float64}` this is also what Julia does.

However, Julia also supports generic linear algebra, allowing you to, for example, work
with matrices and vectors of rational numbers.

#### Rational numbers
julia has rational numbers built in. To construct a rational number, use double forward slashes:

In [31]:
1//2

1//2

#### Example: Rational linear system of equations
The following example shows how linear system of equations with rational elements can be 
solved without promoting to floating point element types. Overflow can easily become a 
problem when working with rational numbers so we use `BigInt`s.

In [32]:
Arational = Matrix{Rational{BigInt}}(rand(1:10, 3, 3))/10

3×3 Array{Rational{BigInt},2}:
 7//10  1//10  3//5
 3//10  1//10  1//2
 9//10  1//1   3//10

In [33]:
x = fill(1, 3)
b = Arational*x

3-element Array{Rational{BigInt},1}:
  7//5
  9//10
 11//5

In [34]:
Arational\b

3-element Array{Rational{BigInt},1}:
 1//1
 1//1
 1//1

In [35]:
lu(Arational)

LU{Rational{BigInt},Array{Rational{BigInt},2}}
L factor:
3×3 Array{Rational{BigInt},2}:
 1//1   0//1   0//1
 7//9   1//1   0//1
 1//3  21//61  1//1
U factor:
3×3 Array{Rational{BigInt},2}:
 9//10    1//1     3//10
 0//1   -61//90   11//30
 0//1     0//1   167//610

#### Exercises

#### 11.1
What are the eigenvalues of matrix A?
```
A = 
[
  140   97   74  168  131
  97  106   89  131   36
   74   89  152  144   71
  168  131  144   54  142
  131   36   71  142   36
]
```
and assign it a variable `A_eigv`

In [1]:
using LinearAlgebra

In [2]:
A = 
[
  140   97   74  168  131
  97  106   89  131   36
   74   89  152  144   71
  168  131  144   54  142
  131   36   71  142   36
]

5×5 Array{Int64,2}:
 140   97   74  168  131
  97  106   89  131   36
  74   89  152  144   71
 168  131  144   54  142
 131   36   71  142   36

In [3]:
A_eigv = eigvals(A)

5-element Array{Float64,1}:
 -128.49322764802145
  -55.887784553056875
   42.7521672793189
   87.16111477514521
  542.4677301466143

In [4]:
@assert A_eigv ==  [-128.49322764802145, -55.887784553056875, 42.7521672793189, 87.16111477514521, 542.4677301466143]

#### 11.2
Create a `Diagonal` matrix from the eigenvalues of `A`.

In [5]:
A_diag = Diagonal(A_eigv)

5×5 Diagonal{Float64,Array{Float64,1}}:
 -128.493     ⋅        ⋅        ⋅         ⋅ 
     ⋅     -55.8878    ⋅        ⋅         ⋅ 
     ⋅        ⋅      42.7522    ⋅         ⋅ 
     ⋅        ⋅        ⋅      87.1611     ⋅ 
     ⋅        ⋅        ⋅        ⋅      542.468

In [8]:
@assert A_diag ==  [-128.493    0.0      0.0      0.0       0.0
    0.0    -55.8878   0.0      0.0       0.0
    0.0      0.0     42.7522   0.0       0.0
    0.0      0.0      0.0     87.1611    0.0
    0.0 0.0      0.0      0.0     542.468]

AssertionError: AssertionError: A_diag == [-128.493 0.0 0.0 0.0 0.0; 0.0 -55.8878 0.0 0.0 0.0; 0.0 0.0 42.7522 0.0 0.0; 0.0 0.0 0.0 87.1611 0.0; 0.0 0.0 0.0 0.0 542.468]

#### 11.3
Create a `LowerTriangluar` matrix from `A` and store it in `A-lowertri`

In [9]:
A_lowertri = LowerTriangular(A)

5×5 LowerTriangular{Int64,Array{Int64,2}}:
 140    ⋅    ⋅    ⋅   ⋅
  97  106    ⋅    ⋅   ⋅
  74   89  152    ⋅   ⋅
 168  131  144   54   ⋅
 131   36   71  142  36

In [10]:
@assert A_lowertri ==  [140    0    0    0   0
  97  106    0    0   0
  74   89  152    0   0
 168  131  144   54   0
 131   36   71  142  36]

### Please let us know how we're doing!
https://tinyurl.com/introJuliaFeedback